SUPERSELECTOR

In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [76]:
def super_selector(dataset, target_col="", selectores={}, hard_voting=[]):
    """
    Selecciona características de un dataframe de features según diferentes métodos de selección.

    Args:
        dataset (pandas.DataFrame): El dataframe de características.
        target_col (str): El nombre de la columna objetivo.
        selectores (dict): Un diccionario con métodos de selección como claves y sus parámetros como valores.
        hard_voting (list): Una lista de características para realizar hard voting.

    Returns:
        dict: Un diccionario que contiene las características seleccionadas por cada método de selección
              y el resultado del hard voting si se especifica.

    Ejemplo:
        selectores = {
            "KBest": 5,
            "FromModel": [RandomForestClassifier(), 5],
            "RFE": [LogisticRegression(), 5, 1]
        }
        result = super_selector(train_set_titanic, target_col="Survived", selectores=selectores, hard_voting=["Pclass", "who", "embarked_S", "fare", "age"])
        print(result)
    """

    result = {}  # Se inicializa un diccionario para almacenar el resultado de la selección de características.

    # Verificar si target_col es válido
    if target_col and target_col in dataset.columns:  # Verifica si se proporcionó un nombre de columna objetivo y si existe en el dataframe.
        target = dataset[target_col]  # Guarda la columna objetivo en 'target'.
        features = dataset.drop(columns=[target_col])  # Elimina la columna objetivo del dataframe de características y guarda el resto en 'features'.
    else:
        target = None  # Si no se proporciona una columna objetivo válida, se establece como 'None'.
        features = dataset.copy()  # Si no se proporciona una columna objetivo válida, se copian todas las características del dataframe.

    # Seleccionar características basadas en selectores
    for selector, params in selectores.items():  # Itera sobre cada método de selección y sus parámetros.
        if selector == "KBest":  # Si el método de selección es 'KBest' (Selección de las mejores características basadas en pruebas univariadas).
            k_best = SelectKBest(score_func=f_classif, k=params)  # Inicializa el selector de las mejores características.
            k_best.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[k_best.get_support()]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

        elif selector == "FromModel":  # Si el método de selección es 'FromModel' (Selección de características basadas en un modelo externo).
            model, threshold = params  # Obtiene el modelo y el umbral de los parámetros.
            if isinstance(threshold, int):  # Comprueba si el umbral es un número entero.
                max_features = threshold  # Si es un número entero, se establece como el número máximo de características.
                threshold = -np.inf  # El umbral se establece en infinito negativo.
            else:
                max_features = None  # Si no es un número entero, el número máximo de características no se limita.
            selector_model = SelectFromModel(model, threshold=threshold, max_features=max_features)  # Inicializa el selector basado en el modelo.
            selector_model.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[selector_model.get_support()]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

        elif selector == "RFE":  # Si el método de selección es 'RFE' (Eliminación recursiva de características).
            model, n_features, step = params  # Obtiene el modelo, el número de características y el paso de los parámetros.
            rfe = RFE(estimator=model, n_features_to_select=n_features, step=step)  # Inicializa el selector de RFE.
            rfe.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[rfe.support_]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

        elif selector == "SFS":  # Si el método de selección es 'SFS' (Selección secuencial hacia adelante).
            model, n_features = params  # Obtiene el modelo y el número de características de los parámetros.
            sfs = SequentialFeatureSelector(model, k_features=n_features, forward=True, floating=False, scoring='accuracy', cv=StratifiedKFold(5))  # Inicializa el selector SFS.
            sfs.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[list(sfs.k_feature_idx_)]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

    # Realizar hard voting
    
    all_features = [result[key] for key in result]  # Combinar todas las listas seleccionadas
    all_features.extend(hard_voting)     # Extender la lista combinada con las características para hard voting
    if all_features: # Verificar si existen características seleccionadas
        voting_counts = Counter([feature for sublist in all_features for feature in sublist if feature in features.columns])   # Co ntar la frecuencia de cada característica en las listas combinadas
        sorted_voting = sorted(voting_counts.items(), key=lambda x: x[1], reverse=True) # Ordenar las características por frecuencia en orden descendente
        hard_voting_result = [feature[0] for feature in sorted_voting[:len(hard_voting)]] # Seleccionar las características más votadas para hard voting
        result['hard_voting'] = hard_voting_result # Agregar el resultado de hard voting al diccionario de resultados
    elif 'hard_voting' in result: # Eliminar el resultado de hard voting si no hay características seleccionadas
        del result['hard_voting']
    # Eliminar 'hard_voting' si está vacío
    if 'hard_voting' in result and not result['hard_voting']:
        del result['hard_voting']

    return result  # Devuelve el resultado que contiene las características seleccionadas por cada método de selección y el resultado del hard voting si se especifica.

In [55]:
titanic_data = pd.read_csv("titanic.csv")

In [56]:
titanic_data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [57]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [58]:
# Preprocesamiento de datos
# Rellenar los valores faltantes en la columna "Age" según el valor en la columna "Who"
mean_age_male = titanic_data.loc[titanic_data['who'] == 'man', 'age'].mean()
mean_age_female = titanic_data.loc[titanic_data['who'] == 'woman', 'age'].mean()
mean_age_child = titanic_data.loc[titanic_data['who'] == 'child', 'age'].mean()

titanic_data.loc[(titanic_data['who'] == 'man') & (titanic_data['age'].isna()), 'age'] = mean_age_male
titanic_data.loc[(titanic_data['who'] == 'woman') & (titanic_data['age'].isna()), 'age'] = mean_age_female
titanic_data.loc[(titanic_data['who'] == 'child') & (titanic_data['age'].isna()), 'age'] = mean_age_child

# Eliminar columnas
titanic_data.drop(['embark_town', 'deck', 'sex', 'class', 'adult_male', 'alive'], axis=1, inplace=True)

# Rellenar NaN en "embarked" con la moda
embarked_mode = titanic_data['embarked'].mode()
titanic_data['embarked'].fillna(embarked_mode, inplace=True)

titanic_data = pd.get_dummies(titanic_data, columns=['embarked'])
titanic_data = titanic_data.replace({False: 0, True: 1})

# Transformar la columna "Who"
titanic_data['who'] = titanic_data['who'].map({'man': 0, 'child': 1, 'woman': 2})

In [59]:
titanic_data

,survived,pclass,age,sibsp,parch,fare,who,alone,embarked_C,embarked_Q,embarked_S
0,0,3,22.0,1,0,7.2500,0,0,0,0,1
1,1,1,38.0,1,0,71.2833,2,0,1,0,0
2,1,3,26.0,0,0,7.9250,2,1,0,0,1
3,1,1,35.0,1,0,53.1000,2,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,1,0,0,1
887,1,1,19.0,0,0,30.0000,2,1,0,0,1
888,0,3,32.0,1,2,23.4500,2,0,0,0,1
889,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [60]:
train_set_titanic, test_set_titanic = train_test_split(titanic_data, test_size = 0.2, random_state = 42)


In [70]:
# Ejemplo de uso
selectores = {
    "KBest": 5,
    "FromModel": [RandomForestClassifier(), 5],
    "RFE": [LogisticRegression(), 5, 1],
    "SFS": [RandomForestClassifier(), 5] 
}
result = super_selector(train_set_titanic, target_col="survived", selectores=selectores, hard_voting=["pclass","who","embarked_S","fare","age"])
result

{'KBest': ['pclass', 'fare', 'who', 'alone', 'embarked_C'],
 'FromModel': ['pclass', 'age', 'sibsp', 'fare', 'who'],
 'RFE': ['pclass', 'sibsp', 'who', 'alone', 'embarked_C'],
 'SFS': ['pclass', 'sibsp', 'who', 'alone', 'embarked_S'],
 'hard_voting': ['pclass', 'who', 'alone', 'sibsp', 'fare']}

Si hard_voting era lista vacia no se incluye:

In [74]:
# Ejemplo de uso
selectores = {
    "KBest": 5,
    "FromModel": [RandomForestClassifier(), 5],
    "RFE": [LogisticRegression(), 5, 1],
    "SFS": [RandomForestClassifier(), 5] 
}
result = super_selector(train_set_titanic, target_col="survived", selectores=selectores, hard_voting=[])
result

{'KBest': ['pclass', 'fare', 'who', 'alone', 'embarked_C'],
 'FromModel': ['pclass', 'age', 'sibsp', 'fare', 'who'],
 'RFE': ['pclass', 'sibsp', 'who', 'alone', 'embarked_C'],
 'SFS': ['pclass', 'sibsp', 'who', 'alone', 'embarked_Q']}